# Handling Missing Data
One of the first steps in any data science workflow is to understand the dataset and to clean it. This is because real world datasets are often very messy and require significant preprocessing before they can be used for subsequent data science tasks such as feature engineering, model training, etc. One of the tasks within data cleaning is to handle with missing data. There are several approaches that can be taken for missing data, such as dropping it, filling with 0's, filling with mean, KNN imputation, etc. In this notebook, we will explore 2 of these imputation techniques, and compare their effectiveness on two sample datasets.

a. The first sample dataset we will use is random numbers, we will generate ~1000 random numbers and perform basic KNN and mean imputation.

b. The second sample dataset we will use is UCI housing dataset, we will use both scaled and non-scaled imputation technique for mean and KNN imputation

# What to try in this notebook?

#### 1. Get a random number generated dataset from kaggle, use one column and create missing (1%, 5%, 10%), scale values, apply KNN, MEAN imputation. Compare the results and compute mean() and var() for the list of differences between org. and Imputed value 


#### 2. Use a housing dataset from UCI, use one column and create missing (1%, 5%, 10%), scale values, apply KNN, MEAN imputation. Compare the results and compute mean() and var() for the list of differences between org. and Imputed value 

Dataset - https://raw.githubusercontent.com/SheshNGupta/datasets/main/train.csv

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# 1.1 Random Numbers dataset

In [3]:
random_dataset = pd.DataFrame({'number': np.random.rand(1000)})

In [5]:
random_dataset.sample(10)

,number
823,0.925249
266,0.077479
959,0.897447
493,0.259423
768,0.193178
105,0.174632
610,0.456349
824,0.688290
968,0.493667
849,0.368834


In [6]:
random_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   number  1000 non-null   float64
dtypes: float64(1)
memory usage: 7.9 KB


In [7]:
random_dataset['number'].describe()

count    1000.000000
mean        0.494461
std         0.286876
min         0.001560
25%         0.252068
50%         0.489302
75%         0.733584
max         0.999815
Name: number, dtype: float64

#### Create 3 col. for numbers for 1%, 5% and 10% missing data

In [8]:
df_number = random_dataset[['number']]
df_number['number_copy_1_percent'] = df_number[['number']]
df_number['number_copy_5_percent'] = df_number[['number']]
df_number['number_copy_10_percent'] = df_number[['number']]
df_number

,number,number_copy_1_percent,number_copy_5_percent,number_copy_10_percent
0,0.438564,0.438564,0.438564,0.438564
1,0.836801,0.836801,0.836801,0.836801
2,0.798077,0.798077,0.798077,0.798077
3,0.269161,0.269161,0.269161,0.269161
4,0.830948,0.830948,0.830948,0.830948
...,...,...,...,...
995,0.920130,0.920130,0.920130,0.920130
996,0.007397,0.007397,0.007397,0.007397
997,0.163360,0.163360,0.163360,0.163360
998,0.553700,0.553700,0.553700,0.553700


#### Check % missing values in this dataframe

In [9]:
def get_percent_missing(dataframe):
    
    percent_missing = dataframe.isnull().sum() * 100 / len(dataframe)
    missing_value_df = pd.DataFrame({'column_name': dataframe.columns,
                                     'percent_missing': percent_missing})
    return missing_value_df

In [10]:
print(get_percent_missing(df_number))

                                   column_name  percent_missing
number                                  number              0.0
number_copy_1_percent    number_copy_1_percent              0.0
number_copy_5_percent    number_copy_5_percent              0.0
number_copy_10_percent  number_copy_10_percent              0.0


#### Create missing helper fn

In [11]:
def create_missing(dataframe, percent, col):
    dataframe.loc[dataframe.sample(frac = percent).index, col] = np.nan

#### Create missing data in each col

In [12]:
create_missing(df_number, 0.01, 'number_copy_1_percent')
create_missing(df_number, 0.05, 'number_copy_5_percent')
create_missing(df_number, 0.1, 'number_copy_10_percent')

#### Check % missing after removing data

In [13]:
print(get_percent_missing(df_number))

                                   column_name  percent_missing
number                                  number              0.0
number_copy_1_percent    number_copy_1_percent              1.0
number_copy_5_percent    number_copy_5_percent              5.0
number_copy_10_percent  number_copy_10_percent             10.0


#### Store the indices of missing rows

In [14]:
# Store Index of NaN values in each coloumns
number_1_idx = list(np.where(df_number['number_copy_1_percent'].isna())[0])
number_5_idx = list(np.where(df_number['number_copy_5_percent'].isna())[0])
number_10_idx = list(np.where(df_number['number_copy_10_percent'].isna())[0])

In [15]:
print(f"Length of number_1_idx is {len(number_1_idx)} and it contains {(len(number_1_idx)/len(df_number['number_copy_1_percent']))*100}% of total data in column | Total rows: {len(df_number['number_copy_1_percent'])}")
print(f"Length of number_5_idx is {len(number_5_idx)} and it contains {(len(number_5_idx)/len(df_number['number_copy_1_percent']))*100}% of total data in column | Total rows: {len(df_number['number_copy_1_percent'])}")
print(f"Length of number_10_idx is {len(number_10_idx)} and it contains {(len(number_10_idx)/len(df_number['number_copy_1_percent']))*100}% of total data in column | Total rows: {len(df_number['number_copy_1_percent'])}")

Length of number_1_idx is 10 and it contains 1.0% of total data in column | Total rows: 1000
Length of number_5_idx is 50 and it contains 5.0% of total data in column | Total rows: 1000
Length of number_10_idx is 100 and it contains 10.0% of total data in column | Total rows: 1000


## What is KNN imputation?
Imputation methodology that works on data that identifies the neighboring points through a measure of distance and the missing values can be estimated using completed values of neighboring observations.

### Perform KNN impute to df_number dataframe

In [16]:
df_number1 = df_number.copy(deep=True)
imputer = KNNImputer(n_neighbors=5)
imputed_number_df = pd.DataFrame(imputer.fit_transform(df_number1), columns = df_number1.columns)


In [17]:
imputed_number_df.sample(10)

,number,number_copy_1_percent,number_copy_5_percent,number_copy_10_percent
701,0.244629,0.244629,0.244629,0.244629
39,0.517202,0.517202,0.517202,0.517202
335,0.100813,0.100813,0.100813,0.100813
204,0.277534,0.277534,0.277534,0.277534
391,0.859032,0.859032,0.857231,0.859032
203,0.252622,0.252622,0.252622,0.252622
144,0.844587,0.844587,0.844587,0.844587
201,0.431603,0.431603,0.431603,0.431603
749,0.848537,0.848537,0.848537,0.848240
497,0.464531,0.464531,0.464531,0.464531


#### Check the % missing data in dataframe now

In [18]:
print(get_percent_missing(imputed_number_df))

                                   column_name  percent_missing
number                                  number              0.0
number_copy_1_percent    number_copy_1_percent              0.0
number_copy_5_percent    number_copy_5_percent              0.0
number_copy_10_percent  number_copy_10_percent              0.0


#### Store the list of differences between org. and Imputed value

In [19]:
# create list of difference bwtween imputed and orginal value

number_diff_1 = []
number_diff_5 = []
number_diff_10 = []
count = 0

for i in number_1_idx:
    count +=1
    diff1 = abs(imputed_number_df['number_copy_1_percent'][i] - df_number1['number'][i])
    number_diff_1.append(diff1)
    

for i in number_5_idx:
    diff5 = abs(imputed_number_df['number_copy_5_percent'][i] - df_number1['number'][i])
    number_diff_5.append(diff5)

for i in number_10_idx:
    diff10 = abs(imputed_number_df['number_copy_10_percent'][i] - df_number1['number'][i])
    number_diff_10.append(diff10)

In [20]:
print(len(number_diff_1))
print(len(number_diff_5))
print(len(number_diff_10))

10
50
100


### Calculate the mean and varience of list of differences KNN

In [21]:
m1 = sum(number_diff_1) / len(number_diff_1)

# calculate variance using a list comprehension
var_res1 = sum((xi - m1) ** 2 for xi in number_diff_1) / len(number_diff_1)

m5 = sum(number_diff_5) / len(number_diff_5)

# calculate variance using a list comprehension
var_res5 = sum((xii - m5) ** 2 for xii in number_diff_5) / len(number_diff_5)


m10 = sum(number_diff_10) / len(number_diff_10)

# calculate variance using a list comprehension
var_res10 = sum((xiii - m10) ** 2 for xiii in number_diff_10) / len(number_diff_10)

print(f"The mean of 1% is {m1} and varience 1% is {var_res1}")
print(f"The mean of 5% is {m5} and varience 5% is {var_res5}")
print(f"The mean of 10% is {m5} and varience 10% is {var_res10}")

The mean of 1% is 0.0005846547543839273 and varience 1% is 2.970798404420463e-07
The mean of 5% is 0.000757031064033434 and varience 5% is 4.329913201182178e-07
The mean of 10% is 0.000757031064033434 and varience 10% is 4.0351965946805086e-07


In [22]:
df_knn_number = pd.DataFrame.from_dict({'1%_number': [m1, var_res1],
 '5%_number': [m5, var_res5],
 '10%_number': [m10, var_res10]}, orient='index')
df_knn_number.columns=['diff. list Mean(KNN)', 'diff. list Var.(KNN)']

## What is Mean imputation?
Mean imputation (MI) is a method in which the mean of the observed values for each variable is computed and the missing values for that variable are imputed by this mean.

### Perform MEAN based imputation

#### Before mean imputation % missing

In [23]:
df_number2 = df_number.copy(deep=True)
print(get_percent_missing(df_number2))

                                   column_name  percent_missing
number                                  number              0.0
number_copy_1_percent    number_copy_1_percent              1.0
number_copy_5_percent    number_copy_5_percent              5.0
number_copy_10_percent  number_copy_10_percent             10.0


In [24]:
df_number2['number_copy_1_percent'] = df_number2['number_copy_1_percent'].fillna(df_number2['number_copy_1_percent'].mean())
df_number2['number_copy_5_percent'] = df_number2['number_copy_5_percent'].fillna(df_number2['number_copy_5_percent'].mean())
df_number2['number_copy_10_percent'] = df_number2['number_copy_10_percent'].fillna(df_number2['number_copy_10_percent'].mean())

#### After mean impute % missing 

In [25]:
print(get_percent_missing(df_number2))

                                   column_name  percent_missing
number                                  number              0.0
number_copy_1_percent    number_copy_1_percent              0.0
number_copy_5_percent    number_copy_5_percent              0.0
number_copy_10_percent  number_copy_10_percent              0.0


In [26]:
df_number2.sample(10)

,number,number_copy_1_percent,number_copy_5_percent,number_copy_10_percent
293,0.583231,0.583231,0.583231,0.583231
461,0.867035,0.867035,0.867035,0.867035
875,0.676228,0.676228,0.676228,0.676228
999,0.771442,0.771442,0.771442,0.771442
75,0.909050,0.909050,0.909050,0.909050
98,0.629583,0.629583,0.629583,0.629583
381,0.181614,0.181614,0.181614,0.181614
592,0.523109,0.523109,0.523109,0.523109
155,0.038074,0.038074,0.038074,0.038074
630,0.869200,0.869200,0.869200,0.869200


#### Create a list of difference -  MEAN

In [27]:
# create list of difference bwtween imputed and orginal value

number_diff_1_mean = []
number_diff_5_mean = []
number_diff_10_mean = []
count = 0

for i in number_1_idx:
    count +=1
    diff1 = abs(df_number2['number_copy_1_percent'][i] - df_number2['number'][i])
    number_diff_1_mean.append(diff1)
    

for i in number_5_idx:
    diff5 = abs(df_number2['number_copy_5_percent'][i] - df_number2['number'][i])
    number_diff_5_mean.append(diff5)

for i in number_10_idx:
    diff10 = abs(df_number2['number_copy_10_percent'][i] - df_number2['number'][i])
    number_diff_10_mean.append(diff10)

In [28]:
print(len(number_diff_1_mean))
print(len(number_diff_5_mean))
print(len(number_diff_10_mean))

10
50
100


### Calculate the mean and var of the list of differences - MEAN Impute

In [29]:
m1 = sum(number_diff_1_mean) / len(number_diff_1_mean)

# calculate variance using a list comprehension
var_res1 = sum((xi - m1) ** 2 for xi in number_diff_1_mean) / len(number_diff_1_mean)

m5 = sum(number_diff_5_mean) / len(number_diff_5_mean)

# calculate variance using a list comprehension
var_res5 = sum((xii - m5) ** 2 for xii in number_diff_5_mean) / len(number_diff_5_mean)


m10 = sum(number_diff_10_mean) / len(number_diff_10_mean)

# calculate variance using a list comprehension
var_res10 = sum((xiii - m10) ** 2 for xiii in number_diff_10_mean) / len(number_diff_10_mean)

print(f"The mean of 1% is {m1} and varience 1% is {var_res1}")
print(f"The mean of 5% is {m5} and varience 5% is {var_res5}")
print(f"The mean of 10% is {m5} and varience 10% is {var_res10}")

The mean of 1% is 0.29595595666774266 and varience 1% is 0.02234691636534702
The mean of 5% is 0.2606794287327926 and varience 5% is 0.017948559982927326
The mean of 10% is 0.2606794287327926 and varience 10% is 0.019225304317791198


In [30]:
df_MI_number = pd.DataFrame.from_dict({'1%_number': [m1, var_res1],
 '5%_number': [m5, var_res5],
 '10%_number': [m10, var_res10]}, orient='index')
df_MI_number.columns=['diff. list Mean(MI)', 'diff. list Var.(MI)']

## KNN and MEAN columns side by side

In [31]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [32]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [33]:
# https://github.com/epmoyer/ipy_table/issues/24

from IPython.core.display import HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

In [34]:
print(number_1_idx[:5])

[103, 272, 302, 441, 542]


In [35]:
compare_1 = imputed_number_df.loc[:, ["number", "number_copy_1_percent"]]
compare_5 = imputed_number_df.loc[:, ["number", "number_copy_5_percent"]]
compare_10 = imputed_number_df.loc[:, ["number", "number_copy_10_percent"]]

In [36]:
compare_1_df =  compare_1.iloc[number_1_idx]
compare_5_df =  compare_5.iloc[number_5_idx]
compare_10_df =  compare_10.iloc[number_10_idx]

In [37]:
compare_1_mean = df_number2.loc[:, ["number", "number_copy_1_percent"]]
compare_5_mean = df_number2.loc[:, ["number", "number_copy_5_percent"]]
compare_10_mean = df_number2.loc[:, ["number", "number_copy_10_percent"]]

In [38]:
compare_1_mean_df =  compare_1_mean.iloc[number_1_idx]
compare_5_mean_df =  compare_5_mean.iloc[number_5_idx]
compare_10_mean_df =  compare_10_mean.iloc[number_10_idx]

#### **number 1% KNN Impute VS number 1% Mean Impute**

In [39]:
multi_table([compare_1_df.head(), compare_1_mean_df.head()])

,number,number_copy_1_percent
103,0.915554,0.915539
272,0.899497,0.899795
302,0.091276,0.090500
441,0.050874,0.050914
542,0.744744,0.744208
,number,number_copy_1_percent
103,0.915554,0.493992
272,0.899497,0.493992
302,0.091276,0.493992
441,0.050874,0.493992


#### **number 5% KNN Impute VS number 5% Mean Impute**

In [40]:
multi_table([compare_5_df.head(), compare_5_mean_df.head()])

,number,number_copy_5_percent
6,0.040451,0.039472
14,0.852026,0.849692
16,0.213343,0.212438
49,0.608203,0.609078
64,0.973574,0.972234
,number,number_copy_5_percent
6,0.040451,0.49266
14,0.852026,0.49266
16,0.213343,0.49266
49,0.608203,0.49266


#### **number 10% KNN Impute VS number 10% Mean Impute**

In [41]:
multi_table([compare_10_df.head(), compare_10_mean_df.head()])

,number,number_copy_10_percent
10,0.205755,0.206019
16,0.213343,0.212724
27,0.738704,0.737446
29,0.322577,0.322495
43,0.403866,0.404988
,number,number_copy_10_percent
10,0.205755,0.50025
16,0.213343,0.50025
27,0.738704,0.50025
29,0.322577,0.50025


# 1.2 Random Numbers dataset Results - KNN and MEAN

In [48]:
multi_table([df_knn_number, df_MI_number])

,diff. list Mean(KNN),diff. list Var.(KNN)
1%_number,0.000585,2.970798e-07
5%_number,0.000757,4.329913e-07
10%_number,0.000661,4.035197e-07
,diff. list Mean(MI),diff. list Var.(MI)
1%_number,0.295956,0.022347
5%_number,0.260679,0.017949
10%_number,0.242477,0.019225


Results : We can see here that KNN performed much better than the mean imputation since KNN will use the method of finding the nearest neighbour. The error in the actual and the imputed value is almost close to zero which signifies that this method is actually predicting and imputing correct values.

# 2.1 Housing Dataset 

In [51]:
housing_data = pd.read_csv('https://raw.githubusercontent.com/SheshNGupta/datasets/main/train.csv')

In [52]:
housing_data.sample(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
740,741,70,RM,60.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,5,2007,WD,Abnorml,132000
1209,1210,20,RL,85.0,10182,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,New,Partial,290000
64,65,60,RL,NaN,9375,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,2,2009,WD,Normal,219500
208,209,60,RL,NaN,14364,Pave,NaN,IR1,Low,AllPub,...,0,NaN,NaN,NaN,0,4,2007,WD,Normal,277000
436,437,50,RM,40.0,4400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,116000
19,20,20,RL,70.0,7560,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2009,COD,Abnorml,139000
1449,1450,180,RM,21.0,1533,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2006,WD,Abnorml,92000
449,450,50,RM,50.0,6000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2007,WD,Normal,120000
1185,1186,50,RL,60.0,9738,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2006,WD,Normal,104900
1023,1024,120,RL,43.0,3182,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,191000


In [53]:
housing_data['SalePrice'].nunique()

663

In [54]:
housing_data['LotArea'].nunique()

1073

In [55]:
housing_data['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [56]:
housing_data['LotArea'].describe()

count      1460.000000
mean      10516.828082
std        9981.264932
min        1300.000000
25%        7553.500000
50%        9478.500000
75%       11601.500000
max      215245.000000
Name: LotArea, dtype: float64

In [57]:
pd.set_option('display.max_rows', None)
print(get_percent_missing(housing_data))

                 column_name  percent_missing
Id                        Id         0.000000
MSSubClass        MSSubClass         0.000000
MSZoning            MSZoning         0.000000
LotFrontage      LotFrontage        17.739726
LotArea              LotArea         0.000000
Street                Street         0.000000
Alley                  Alley        93.767123
LotShape            LotShape         0.000000
LandContour      LandContour         0.000000
Utilities          Utilities         0.000000
LotConfig          LotConfig         0.000000
LandSlope          LandSlope         0.000000
Neighborhood    Neighborhood         0.000000
Condition1        Condition1         0.000000
Condition2        Condition2         0.000000
BldgType            BldgType         0.000000
HouseStyle        HouseStyle         0.000000
OverallQual      OverallQual         0.000000
OverallCond      OverallCond         0.000000
YearBuilt          YearBuilt         0.000000
YearRemodAdd    YearRemodAdd      

#### Using Sale price coloumn for KNN and MEAN imputation task

#### Non Scaled dataframe Sale Price - take first 1000 rows

In [58]:
df_saleprice = housing_data[['SalePrice']][:1000]
df_saleprice['sp_copy_1_percent'] = df_saleprice[['SalePrice']]
df_saleprice['sp_copy_5_percent'] = df_saleprice[['SalePrice']]
df_saleprice['sp_copy_10_percent'] = df_saleprice[['SalePrice']]
df_saleprice.head()

,SalePrice,sp_copy_1_percent,sp_copy_5_percent,sp_copy_10_percent
0,208500,208500,208500,208500
1,181500,181500,181500,181500
2,223500,223500,223500,223500
3,140000,140000,140000,140000
4,250000,250000,250000,250000


In [59]:
len(df_saleprice)

1000

#### Scaled Dataframe SalePrice - take first 1000 rows

In [60]:
scaler = MinMaxScaler()
df_saleprice_scaled = df_saleprice.copy(deep=True)
df_saleprice_scaled = pd.DataFrame(scaler.fit_transform(df_saleprice_scaled), columns = df_saleprice_scaled.columns)
df_saleprice_scaled.head()

,SalePrice,sp_copy_1_percent,sp_copy_5_percent,sp_copy_10_percent
0,0.241078,0.241078,0.241078,0.241078
1,0.203583,0.203583,0.203583,0.203583
2,0.261908,0.261908,0.261908,0.261908
3,0.145952,0.145952,0.145952,0.145952
4,0.298709,0.298709,0.298709,0.298709


#### Check % missing values in this dataframe

In [61]:
print(get_percent_missing(df_saleprice))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              0.0
sp_copy_5_percent    sp_copy_5_percent              0.0
sp_copy_10_percent  sp_copy_10_percent              0.0


#### Create 1%, 5% and 10% missing data

In [62]:
create_missing(df_saleprice, 0.01, 'sp_copy_1_percent')
create_missing(df_saleprice, 0.05, 'sp_copy_5_percent')
create_missing(df_saleprice, 0.1, 'sp_copy_10_percent')

In [63]:
create_missing(df_saleprice_scaled, 0.01, 'sp_copy_1_percent')
create_missing(df_saleprice_scaled, 0.05, 'sp_copy_5_percent')
create_missing(df_saleprice_scaled, 0.1, 'sp_copy_10_percent')

#### With/Without scaling dataframe missing values check

In [64]:
print(get_percent_missing(df_saleprice))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              1.0
sp_copy_5_percent    sp_copy_5_percent              5.0
sp_copy_10_percent  sp_copy_10_percent             10.0


In [65]:
print(get_percent_missing(df_saleprice_scaled))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              1.0
sp_copy_5_percent    sp_copy_5_percent              5.0
sp_copy_10_percent  sp_copy_10_percent             10.0


In [66]:
df_saleprice['sp_copy_1_percent'].isna().sum()

10

#### Store indices of missing values

In [67]:
# Store Index of NaN values in each coloumns
sp_1_idx = list(np.where(df_saleprice['sp_copy_1_percent'].isna())[0])
sp_5_idx = list(np.where(df_saleprice['sp_copy_5_percent'].isna())[0])
sp_10_idx = list(np.where(df_saleprice['sp_copy_10_percent'].isna())[0])

In [68]:
print(len(sp_1_idx))
print(len(sp_5_idx))
print(len(sp_10_idx))

10
50
100


In [69]:
print(f"Length of sp_1_idx is {len(sp_1_idx)} and it contains {(len(sp_1_idx)/len(df_saleprice['sp_copy_1_percent']))*100}% of total data in column | Total rows: {len(df_saleprice['sp_copy_1_percent'])}")
print(f"Length of sp_5_idx is {len(sp_5_idx)} and it contains {(len(sp_5_idx)/len(df_saleprice['sp_copy_5_percent']))*100}% of total data in column | Total rows: {len(df_saleprice['sp_copy_1_percent'])}")
print(f"Length of sp_10_idx is {len(sp_10_idx)} and it contains {(len(sp_10_idx)/len(df_saleprice['sp_copy_10_percent']))*100}% of total data in column | Total rows: {len(df_saleprice['sp_copy_1_percent'])}")

Length of sp_1_idx is 10 and it contains 1.0% of total data in column | Total rows: 1000
Length of sp_5_idx is 50 and it contains 5.0% of total data in column | Total rows: 1000
Length of sp_10_idx is 100 and it contains 10.0% of total data in column | Total rows: 1000


### Perform KNN to df_saleprice and df_saleprice_scaled dataframe

In [70]:
df_saleprice1 = df_saleprice.copy(deep=True)
imputer = KNNImputer(n_neighbors=5)
imputed_saleprice_df = pd.DataFrame(imputer.fit_transform(df_saleprice1), columns = df_saleprice1.columns)

In [71]:
df_saleprice_scaled1 = df_saleprice_scaled.copy(deep=True)
imputer = KNNImputer(n_neighbors=5)
imputed_saleprice_scaled_df = pd.DataFrame(imputer.fit_transform(df_saleprice_scaled1), columns = df_saleprice_scaled1.columns)

In [72]:
imputed_saleprice_df.head()

,SalePrice,sp_copy_1_percent,sp_copy_5_percent,sp_copy_10_percent
0,208500.0,208500.0,208500.0,208500.0
1,181500.0,181500.0,181500.0,181500.0
2,223500.0,223500.0,223500.0,223500.0
3,140000.0,140000.0,140000.0,140000.0
4,250000.0,250000.0,250000.0,250000.0


In [73]:
imputed_saleprice_scaled_df.head()

,SalePrice,sp_copy_1_percent,sp_copy_5_percent,sp_copy_10_percent
0,0.241078,0.241078,0.241078,0.241078
1,0.203583,0.203583,0.203583,0.203583
2,0.261908,0.261908,0.261908,0.261908
3,0.145952,0.145952,0.145952,0.145952
4,0.298709,0.298709,0.298709,0.298709


#### Check % missing in saleprice and saleprice_scaled DF

In [74]:
print(get_percent_missing(imputed_saleprice_df))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              0.0
sp_copy_5_percent    sp_copy_5_percent              0.0
sp_copy_10_percent  sp_copy_10_percent              0.0


In [75]:
print(get_percent_missing(imputed_saleprice_scaled_df))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              0.0
sp_copy_5_percent    sp_copy_5_percent              0.0
sp_copy_10_percent  sp_copy_10_percent              0.0


#### Store the list of disfferences between Org. and Imputed Value

In [76]:
# create list of difference bwtween imputed and orginal value

sp_diff_1 = []
sp_diff_5 = []
sp_diff_10 = []
count = 0

for i in sp_1_idx:
    count +=1
    diff1 = abs(imputed_saleprice_df['sp_copy_1_percent'][i] - imputed_saleprice_df['SalePrice'][i])
    sp_diff_1.append(diff1)
    

for i in sp_5_idx:
    diff5 = abs(imputed_saleprice_df['sp_copy_5_percent'][i] - imputed_saleprice_df['SalePrice'][i])
    sp_diff_5.append(diff5)

for i in sp_10_idx:
    diff10 = abs(imputed_saleprice_df['sp_copy_10_percent'][i] - imputed_saleprice_df['SalePrice'][i])
    sp_diff_10.append(diff10)

In [77]:
print(len(sp_diff_1))
print(len(sp_diff_5))
print(len(sp_diff_10))

10
50
100


In [78]:
# create list of difference bwtween imputed and orginal value

sp_scaled_diff_1 = []
sp_scaled_diff_5 = []
sp_scaled_diff_10 = []
count = 0

for i in sp_1_idx:
    count +=1
    diff1 = abs(imputed_saleprice_scaled_df['sp_copy_1_percent'][i] - imputed_saleprice_scaled_df['SalePrice'][i])
    sp_scaled_diff_1.append(diff1)
    

for i in sp_5_idx:
    diff5 = abs(imputed_saleprice_scaled_df['sp_copy_5_percent'][i] - imputed_saleprice_scaled_df['SalePrice'][i])
    sp_scaled_diff_5.append(diff5)

for i in sp_10_idx:
    diff10 = abs(imputed_saleprice_scaled_df['sp_copy_10_percent'][i] - imputed_saleprice_scaled_df['SalePrice'][i])
    sp_scaled_diff_10.append(diff10)

In [79]:
print(len(sp_scaled_diff_1))
print(len(sp_scaled_diff_5))
print(len(sp_scaled_diff_10))

10
50
100


In [80]:
sp_scaled_diff_1[:5]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [81]:
sp_diff_1[:5]

[0.0, 100.0, 20.0, 0.0, 780.0]

#### Calculate the mean and var of list of diff. KNN - SalePrice

In [82]:
m1 = sum(sp_diff_1) / len(sp_diff_1)

# calculate variance using a list comprehension
var_res1 = sum((xi - m1) ** 2 for xi in sp_diff_1) / len(sp_diff_1)

m5 = sum(sp_diff_5) / len(sp_diff_5)

# calculate variance using a list comprehension
var_res5 = sum((xii - m5) ** 2 for xii in sp_diff_5) / len(sp_diff_5)


m10 = sum(sp_diff_10) / len(sp_diff_10)

# calculate variance using a list comprehension
var_res10 = sum((xiii - m10) ** 2 for xiii in sp_diff_10) / len(sp_diff_10)

print(f"The mean of 1% is {m1} and varience 1% is {var_res1}")
print(f"The mean of 5% is {m5} and varience 5% is {var_res5}")
print(f"The mean of 10% is {m5} and varience 10% is {var_res10}")

The mean of 1% is 105.0 and varience 1% is 52105.0
The mean of 5% is 163.0120000000001 and varience 5% is 46018.96385599976
The mean of 10% is 163.0120000000001 and varience 10% is 3667553.3671999993


In [83]:
df_knn_saleprice = pd.DataFrame.from_dict({'1%_saleprice': [m1, var_res1],
 '5%_saleprice': [m5, var_res5],
 '10%_saleprice': [m10, var_res10]}, orient='index')
df_knn_saleprice.columns=['diff. list Mean(KNN)', 'diff. list Var.(KNN)']

In [84]:
df_knn_saleprice

,diff. list Mean(KNN),diff. list Var.(KNN)
1%_saleprice,105.000,5.210500e+04
5%_saleprice,163.012,4.601896e+04
10%_saleprice,470.800,3.667553e+06


#### Calculate the mean and var of list of diff. KNN - SalePrice scaled

In [85]:
m1 = sum(sp_scaled_diff_1) / len(sp_scaled_diff_1)

# calculate variance using a list comprehension
var_res1 = sum((xi - m1) ** 2 for xi in sp_scaled_diff_1) / len(sp_scaled_diff_1)

m5 = sum(sp_scaled_diff_5) / len(sp_scaled_diff_5)

# calculate variance using a list comprehension
var_res5 = sum((xii - m5) ** 2 for xii in sp_scaled_diff_5) / len(sp_scaled_diff_5)


m10 = sum(sp_scaled_diff_10) / len(sp_scaled_diff_10)

# calculate variance using a list comprehension
var_res10 = sum((xiii - m10) ** 2 for xiii in sp_scaled_diff_10) / len(sp_scaled_diff_10)

print(f"The mean of 1% is {m1} and varience 1% is {var_res1}")
print(f"The mean of 5% is {m5} and varience 5% is {var_res5}")
print(f"The mean of 10% is {m5} and varience 10% is {var_res10}")

The mean of 1% is 0.0 and varience 1% is 0.0
The mean of 5% is 1.2498264129982007e-05 and varience 5% is 7.654123706876951e-09
The mean of 10% is 1.2498264129982007e-05 and varience 10% is 2.9738417673284677e-06


In [86]:
df_knn_saleprice_scaled = pd.DataFrame.from_dict({'1%_saleprice': [m1, var_res1],
 '5%_saleprice': [m5, var_res5],
 '10%_saleprice': [m10, var_res10]}, orient='index')
df_knn_saleprice_scaled.columns=['diff. list Mean(KNN) scaled', 'diff. list Var.(KNN) scaled']

In [87]:
df_knn_saleprice_scaled

,diff. list Mean(KNN) scaled,diff. list Var.(KNN) scaled
1%_saleprice,0.000000,0.000000e+00
5%_saleprice,0.000012,7.654124e-09
10%_saleprice,0.000265,2.973842e-06


### Perform MEAN imputation

In [88]:
df_saleprice2 = df_saleprice.copy(deep=True)
df_saleprice_scaled2 = df_saleprice_scaled.copy(deep=True)

In [89]:
print(get_percent_missing(df_saleprice2))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              1.0
sp_copy_5_percent    sp_copy_5_percent              5.0
sp_copy_10_percent  sp_copy_10_percent             10.0


In [90]:
print(get_percent_missing(df_saleprice_scaled2))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              1.0
sp_copy_5_percent    sp_copy_5_percent              5.0
sp_copy_10_percent  sp_copy_10_percent             10.0


#### Impute Mean values in missing for saleprice and saleprice_scaled

In [91]:
df_saleprice2['sp_copy_1_percent'] = df_saleprice2['sp_copy_1_percent'].fillna(df_saleprice2['sp_copy_1_percent'].mean())
df_saleprice2['sp_copy_5_percent'] = df_saleprice2['sp_copy_5_percent'].fillna(df_saleprice2['sp_copy_5_percent'].mean())
df_saleprice2['sp_copy_10_percent'] = df_saleprice2['sp_copy_10_percent'].fillna(df_saleprice2['sp_copy_10_percent'].mean())

In [92]:
df_saleprice_scaled2['sp_copy_1_percent'] = df_saleprice_scaled2['sp_copy_1_percent'].fillna(df_saleprice_scaled2['sp_copy_1_percent'].mean())
df_saleprice_scaled2['sp_copy_5_percent'] = df_saleprice_scaled2['sp_copy_5_percent'].fillna(df_saleprice_scaled2['sp_copy_5_percent'].mean())
df_saleprice_scaled2['sp_copy_10_percent'] = df_saleprice_scaled2['sp_copy_10_percent'].fillna(df_saleprice_scaled2['sp_copy_10_percent'].mean())

#### After MEAN imputation - Saleprice and saleprice scaled

In [93]:
print(get_percent_missing(df_saleprice2))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              0.0
sp_copy_5_percent    sp_copy_5_percent              0.0
sp_copy_10_percent  sp_copy_10_percent              0.0


In [94]:
print(get_percent_missing(df_saleprice_scaled2))

                           column_name  percent_missing
SalePrice                    SalePrice              0.0
sp_copy_1_percent    sp_copy_1_percent              0.0
sp_copy_5_percent    sp_copy_5_percent              0.0
sp_copy_10_percent  sp_copy_10_percent              0.0


In [95]:
df_saleprice2.sample(5)

,SalePrice,sp_copy_1_percent,sp_copy_5_percent,sp_copy_10_percent
436,116000,116000.0,116000.0,116000.000000
21,139400,139400.0,139400.0,139400.000000
618,314813,314813.0,314813.0,314813.000000
207,141000,141000.0,141000.0,182369.783333
366,159000,159000.0,159000.0,159000.000000


In [96]:
df_saleprice_scaled2.sample(5)

,SalePrice,sp_copy_1_percent,sp_copy_5_percent,sp_copy_10_percent
457,0.307041,0.307041,0.307041,0.201890
876,0.135190,0.135190,0.135190,0.135190
361,0.152895,0.152895,0.152895,0.152895
682,0.191779,0.191779,0.191779,0.201890
523,0.208096,0.208096,0.208096,0.208096


#### Create List of differences for saleprice and saleprice_scaled Dataframes

In [97]:
# create list of difference bwtween imputed and orginal value

sp_mean_diff_1 = []
sp_mean_diff_5 = []
sp_mean_diff_10 = []
count = 0

for i in sp_1_idx:
    count +=1
    diff1 = abs(df_saleprice2['sp_copy_1_percent'][i] - df_saleprice2['SalePrice'][i])
    sp_mean_diff_1.append(diff1)
    

for i in sp_5_idx:
    diff5 = abs(df_saleprice2['sp_copy_5_percent'][i] - df_saleprice2['SalePrice'][i])
    sp_mean_diff_5.append(diff5)

for i in sp_10_idx:
    diff10 = abs(df_saleprice2['sp_copy_10_percent'][i] - df_saleprice2['SalePrice'][i])
    sp_mean_diff_10.append(diff10)

In [98]:
print(len(sp_mean_diff_1))
print(len(sp_mean_diff_5))
print(len(sp_mean_diff_10))

10
50
100


In [99]:
# create list of difference bwtween imputed and orginal value

sp_scaled_mean_diff_1 = []
sp_scaled_mean_diff_5 = []
sp_scaled_mean_diff_10 = []
count = 0

for i in sp_1_idx:
    count +=1
    diff1 = abs(df_saleprice_scaled2['sp_copy_1_percent'][i] - df_saleprice_scaled2['SalePrice'][i])
    sp_scaled_mean_diff_1.append(diff1)
    

for i in sp_5_idx:
    diff5 = abs(df_saleprice_scaled2['sp_copy_5_percent'][i] - df_saleprice_scaled2['SalePrice'][i])
    sp_scaled_mean_diff_5.append(diff5)

for i in sp_10_idx:
    diff10 = abs(df_saleprice_scaled2['sp_copy_10_percent'][i] - df_saleprice_scaled2['SalePrice'][i])
    sp_scaled_mean_diff_10.append(diff10)

In [100]:
print(len(sp_scaled_mean_diff_1))
print(len(sp_scaled_mean_diff_5))
print(len(sp_scaled_mean_diff_10))

10
50
100


#### Calculate mean and var of list of diff. - MEAN impute SalePrice

In [101]:
m1 = sum(sp_mean_diff_1) / len(sp_mean_diff_1)

# calculate variance using a list comprehension
var_res1 = sum((xi - m1) ** 2 for xi in sp_mean_diff_1) / len(sp_mean_diff_1)

m5 = sum(sp_mean_diff_5) / len(sp_mean_diff_5)

# calculate variance using a list comprehension
var_res5 = sum((xii - m5) ** 2 for xii in sp_mean_diff_5) / len(sp_mean_diff_5)


m10 = sum(sp_mean_diff_10) / len(sp_mean_diff_10)

# calculate variance using a list comprehension
var_res10 = sum((xiii - m10) ** 2 for xiii in sp_mean_diff_10) / len(sp_mean_diff_10)

print(f"The mean of 1% is {m1} and varience 1% is {var_res1}")
print(f"The mean of 5% is {m5} and varience 5% is {var_res5}")
print(f"The mean of 10% is {m5} and varience 10% is {var_res10}")

The mean of 1% is 47198.61696969698 and varience 1% is 634546571.3543438
The mean of 5% is 54438.20686315788 and varience 5% is 1768876209.3358026
The mean of 10% is 54438.20686315788 and varience 10% is 2875290913.3009353


In [102]:
df_mean_saleprice = pd.DataFrame.from_dict({'1%_saleprice': [m1, var_res1],
 '5%_saleprice': [m5, var_res5],
 '10%_saleprice': [m10, var_res10]}, orient='index')
df_mean_saleprice.columns=['diff. list Mean(MI)', 'diff. list Var.(MI)']

In [103]:
df_mean_saleprice

,diff. list Mean(MI),diff. list Var.(MI)
1%_saleprice,47198.616970,6.345466e+08
5%_saleprice,54438.206863,1.768876e+09
10%_saleprice,58045.636667,2.875291e+09


#### Calculate mean and var of list of diff. - MEAN impute SalePrice scaled

In [104]:
m1 = sum(sp_scaled_mean_diff_1) / len(sp_scaled_mean_diff_1)

# calculate variance using a list comprehension
var_res1 = sum((xi - m1) ** 2 for xi in sp_scaled_mean_diff_1) / len(sp_scaled_mean_diff_1)

m5 = sum(sp_scaled_mean_diff_5) / len(sp_scaled_mean_diff_5)

# calculate variance using a list comprehension
var_res5 = sum((xii - m5) ** 2 for xii in sp_scaled_mean_diff_5) / len(sp_scaled_mean_diff_5)


m10 = sum(sp_scaled_mean_diff_10) / len(sp_scaled_mean_diff_10)

# calculate variance using a list comprehension
var_res10 = sum((xiii - m10) ** 2 for xiii in sp_scaled_mean_diff_10) / len(sp_scaled_mean_diff_10)

print(f"The mean of 1% is {m1} and varience 1% is {var_res1}")
print(f"The mean of 5% is {m5} and varience 5% is {var_res5}")
print(f"The mean of 10% is {m5} and varience 10% is {var_res10}")

The mean of 1% is 0.0 and varience 1% is 0.0
The mean of 5% is 0.0016175777048509216 and varience 5% is 5.557201947380946e-05
The mean of 10% is 0.0016175777048509216 and varience 10% is 0.004250732648521598


In [105]:
df_mean_saleprice_scaled = pd.DataFrame.from_dict({'1%_saleprice_scaled': [m1, var_res1],
 '5%_saleprice_scaled': [m5, var_res5],
 '10%_saleprice_scaled': [m10, var_res10]}, orient='index')
df_mean_saleprice_scaled.columns=['diff. list Mean(MI) scaled', 'diff. list Var.(MI) scaled']

In [106]:
df_mean_saleprice_scaled

,diff. list Mean(MI) scaled,diff. list Var.(MI) scaled
1%_saleprice_scaled,0.000000,0.000000
5%_saleprice_scaled,0.001618,0.000056
10%_saleprice_scaled,0.018922,0.004251


# 2.2 Housing Data Results - KNN and MEAN

In [107]:
multi_table([df_knn_saleprice, df_knn_saleprice_scaled, df_mean_saleprice, df_mean_saleprice_scaled])

,diff. list Mean(KNN),diff. list Var.(KNN)
1%_saleprice,105.000,5.210500e+04
5%_saleprice,163.012,4.601896e+04
10%_saleprice,470.800,3.667553e+06
,diff. list Mean(KNN) scaled,diff. list Var.(KNN) scaled
1%_saleprice,0.000000,0.000000e+00
5%_saleprice,0.000012,7.654124e-09
10%_saleprice,0.000265,2.973842e-06
,diff. list Mean(MI),diff. list Var.(MI)
1%_saleprice,47198.616970,6.345466e+08
5%_saleprice,54438.206863,1.768876e+09


Result: Another takeaway here is that if we use scaling before performing the imputation, the imputation works much better and accuratly. Although the mean imputation provided less accurate results as compared to the KNN imputation, but the accuracy of the imputed values are still better if we use scaling than not using it. KNN imputation on other hand did perform better than mean imputation, however the results are much better if we use scaled dataset.

# Conclusion
1. KNN imputation is performing better than mean imputation
2. If we use scaled dataset as compared to non scaled dataset, the results are even better (almost close to perfect!)